In [19]:
import tensorflow as tf
import numpy as np
from PIL import Image

In [29]:
import os

IMAGE_SIZE = 28
X_SHAPE = [128, 128, 3]
Y_SHAPE = [62]

def load_images_and_labels_from_partition_file(file_path):
    images = []
    labels = []
    
    with open(file_path, 'r') as file:
        for line in file:
            image_path = line.strip()
            label = os.path.basename(os.path.dirname(image_path))  # Extract label from the last directory
            label = int(label)  # Convert label to an integer
            
            image = Image.open("/Users/janezbozic/faks/Magistrska/fl-testbed/code/examples/femnist_example/" + image_path)
            image = image.resize((IMAGE_SIZE, IMAGE_SIZE))  # Resize if needed
            image_array = np.array(image) / 255.0  # Normalize pixel values
            
            images.append(image_array)
            labels.append(label)
    
    return np.array(images), np.array(labels)

# Specify the paths to your train and test files
train_file_path = '/Users/janezbozic/faks/Magistrska/fl-testbed/code/examples/femnist_example/data/partition_0_train.txt'
test_file_path = '/Users/janezbozic/faks/Magistrska/fl-testbed/code/examples/femnist_example/data/partition_0_test.txt'

# Load and preprocess training data
train_images, train_labels = load_images_and_labels_from_partition_file(train_file_path)
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, num_classes=62)

# Load and preprocess testing data
test_images, test_labels = load_images_and_labels_from_partition_file(test_file_path)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, num_classes=62)

# Build the model
model = tf.keras.Sequential([
            tf.keras.Input(shape=tuple(X_SHAPE)),
            tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2), strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(2048, activation='relu'),
            tf.keras.layers.Dense(62, activation="softmax"),
        ])

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

# Train the model
model.fit(train_images, train_labels_onehot, epochs=10, validation_data=(test_images, test_labels_onehot))

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 28, 28, 32)        2432      
                                                                 
 max_pooling2d_30 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_31 (Conv2D)          (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_31 (MaxPooli  (None, 7, 7, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_15 (Flatten)        (None, 3136)              0         
                                                                 
 dense_30 (Dense)            (None, 2048)            